In [376]:
import pandas as pd

In [377]:
train = pd.read_csv('train.csv', index_col='_id')

In [378]:
train.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,target
_id,,,,,,,,,,,,,,,,,,,,,
df7489733b004bbe40d3d37b34f82419,54,technician,married,professional.course,unknown,no,no,cellular,aug,tue,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
905a0b9a5456ee962223033473666be3,36,services,single,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
02d69c07ec6d68e31d641edd45ce61cd,53,blue-collar,married,basic.4y,no,yes,no,cellular,aug,mon,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,0
e492fbe0cccaf67cdb5c0944dbc8a167,34,blue-collar,married,basic.9y,no,yes,yes,telephone,may,wed,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
ac2dc6f5835bdea8e71dd99b65208d07,29,services,single,university.degree,no,yes,no,telephone,aug,fri,...,1,999,0,nonexistent,-1.7,94.027,-38.3,0.890,4991.6,0


In [379]:
y = train['target']
train = train.drop(['target','month','day_of_week'], axis=1) # убрал месяц и день недели так как они 
                                                            # никакого влияния на результат иметь не должны

In [380]:
tr = pd.get_dummies(train)
tr = tr.drop(['contact_telephone'], axis=1)

In [381]:
from sklearn.cross_validation import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(tr, y, test_size=0.3, random_state=42)

In [382]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [383]:
dtc = DecisionTreeClassifier(max_depth=5, criterion = 'entropy')
dtc.fit(xTrain, yTrain)

knn = KNeighborsClassifier(n_neighbors=111) 
knn.fit(xTrain, yTrain)

lr = LogisticRegression()
lr.fit(xTrain, yTrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [384]:
from sklearn.metrics import roc_auc_score

In [385]:
roc_auc_score(yTest, dtc.predict_proba(xTest)[:,1])

0.9279925832493434

In [386]:
roc_auc_score(yTest, knn.predict_proba(xTest)[:,1]) # n_neighbors=111 дает наилучший результат 
                                                    # получено методом тыка

0.9285551038180813

In [387]:
roc_auc_score(yTest, lr.predict_proba(xTest)[:,1])

0.9268587865766995

In [268]:
from sklearn.cross_validation import cross_val_score

dtc1 = DecisionTreeClassifier(max_depth=5, criterion = 'entropy')
cross_val_score(
    dtc1,
    tr,
    y,
    scoring='roc_auc',
    cv=10
)

array([0.93257383, 0.92668102, 0.93577398, 0.94085531, 0.93199934,
       0.91825848, 0.94443627, 0.9368559 , 0.92419869, 0.93071273])

In [388]:
import sys
import time
def show_progress(value):             # делаю простой прогресс бар
    sys.stderr.write('%d\r' % value)

In [280]:
import numpy as nm
mx = 0
depth = 0
for i in range(500):
    dtc1 = DecisionTreeClassifier(max_depth=i+1, criterion = 'entropy')
    arr = cross_val_score(
        dtc1,
        tr,
        y,
        scoring='roc_auc',
        cv=10
    )
    m = nm.mean(arr)
    if mx < m: 
        mx = m
        depth = i+1
    show_progress(i+1)
        
print(mx, depth)  # выводим лучший max_depth для дерева принятия решений

0.9353036379529943 6


In [391]:
mx = 0
depth = 0
for i in range(100):
    knn1 = KNeighborsClassifier(n_neighbors=i+1) 
    arr = cross_val_score(
        knn1,
        tr,
        y,
        scoring='roc_auc',
        cv=10
    )
    m = nm.mean(arr)
    if mx < m: 
        mx = m
        depth = i+1
    show_progress(i+1)
        
print(mx, depth)  # выводим лучший n_neighbors для k ближайших соседей

0.9291817902506038 58


In [284]:
lr1 = LogisticRegression()
arr = cross_val_score(
    lr1,
    tr,
    y,
    scoring='roc_auc',
    cv=10
)
nm.mean(arr)

0.922036499236112

In [285]:
# выходит лучший результат показало дерево принятия решений с глубиной 6
# а теперь во-первых обучим на полном наборе train-даных 
# а уже во вторых сделаем требуемый прогноз 

In [370]:
train = pd.read_csv('train.csv', index_col='_id')
y = train['target']
train = train.drop(['target','month','day_of_week'], axis=1)
tr = pd.get_dummies(train)
tr = tr.drop(['contact_telephone'], axis=1)

test = pd.read_csv('test.csv', index_col='_id')
test = test.drop(['month','day_of_week'], axis=1)
tst = pd.get_dummies(test)
tst = tst.drop(['contact_telephone'], axis=1)

In [371]:
dtc_tst = DecisionTreeClassifier(max_depth=6, criterion = 'entropy')
dtc_tst.fit(tr, y)

lr_tst = LogisticRegression()
lr_tst.fit(tr, y)

knn_tst = KNeighborsClassifier(n_neighbors=77) 
knn_tst.fit(tr, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=77, p=2,
           weights='uniform')

In [372]:
predict_dtc = dtc_tst.predict(tst)
predict_lr = lr_tst.predict(tst)
predict_knn = knn_tst.predict(tst)

In [373]:
#tst['target'] = predict_dtc
#tst['target'] = predict_lr
tst['target'] = predict_knn

In [374]:
#result_dtc = tst[['target']]
#result_lr = tst[['target']]
result_knn = tst[['target']]

In [375]:
#result_dtc.to_csv('result_dtc.csv')
#result_lr.to_csv('result_lr.csv')
result_knn.to_csv('result_knn.csv')